# Fill up
This is another strategy trying to solve the problem of 4.4. The first interval is [pc − a, pc + a]. The second interval is [pc, pc + a] or [pc − a, pc], depending on which asset is leftover.

Import code dependencies

In [ ]:
from typing import List

import pandas as pd
from matplotlib.pylab import plt
import matplotlib.dates as mdates

from demeter import MarketInfo
from demeter.broker import AccountStatus
from demeter.metrics import performance_metrics

Set matplotlib to show formatted account status

In [2]:
def plotter(account_status_list: List[AccountStatus]):
    net_value_ts = [status.net_value for status in account_status_list]
    time_ts = [status.timestamp for status in account_status_list]
    plt.plot(time_ts, net_value_ts)
    plt.show()

Make plot about price and account value / position net value

In [3]:
def plot_position_return_decomposition(account_status: pd.DataFrame, price: pd.Series, market: MarketInfo):
    fig, value_ax = plt.subplots()
    day = mdates.DayLocator(interval=2)

    price_ax = value_ax.twinx()
    price_ax.xaxis.set_major_locator(day)
    price_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    value_ax.set_xlabel('time')
    value_ax.set_ylabel('value', color='g')
    price_ax.set_ylabel('price', color='b')

    net_value_ts = list(account_status.net_value)
    time_ts = list(account_status.index)
    price_ts = list(price)

    value_in_position = account_status[market.name]["net_value"]
    value_in_account = account_status["tokens"]["USDC"] + account_status["tokens"]["ETH"] * price

    value_ax.plot(time_ts, net_value_ts, 'g-', label="net value")
    value_ax.plot(time_ts, value_in_position, 'r-', label="value in get_position")
    value_ax.plot(time_ts, value_in_account, 'b-', label="value in broker account")
    price_ax.plot(time_ts, price_ts, 'y-', label="price")
    fig.legend()
    fig.show()

Add dependence about run Actuator

In [4]:
from datetime import date, timedelta
from typing import Dict
from _decimal import Decimal
import pandas as pd

from demeter import TokenInfo, Actuator, ChainType, MarketInfo, Strategy, PeriodTrigger, RowData
from demeter.uniswap import UniLpMarket, UniV3Pool

Set pandas output format

In [5]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Full up strategy with code to add liquidity and use up all token at some side of price

In [6]:
class FillUpStrategy(Strategy):
    def __init__(self, a=10):
        super().__init__()
        self.a = a

    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        init_price = lp_market.market_status.data.price

        lp_market.even_rebalance(init_price)  # rebalance all reserve token#
        lp_market.add_liquidity(init_price - self.a, init_price + self.a)
        if self.broker.assets[market.base_token].balance > 0:
            lp_market.add_liquidity(init_price - self.a, init_price)
        else:
            lp_market.add_liquidity(init_price, init_price + self.a)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))

    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        lp_market.add_liquidity(row_data.prices[eth.name] - self.a, row_data.prices[eth.name] + self.a)
        if self.broker.assets[market.base_token].balance > 0:
            lp_market.add_liquidity(row_data.prices[eth.name] - self.a, row_data.prices[eth.name])
        else:
            lp_market.add_liquidity(row_data.prices[eth.name], row_data.prices[eth.name] + self.a)



Main logic to run Actuator, init two token and market with key "market1"

In [7]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 2000)
broker.set_balance(eth, 0)

actuator.strategy = FillUpStrategy(200)

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()  # run test

2024-07-22 16:24:15,055 - INFO - start load files from 2023-08-13 to 2023-08-17...
2024-07-22 16:24:15,123 - INFO - load file complete, preparing...
2024-07-22 16:24:15,319 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [8]:
actuator.run()
print({k: round(v, 5) for k, v in performance_metrics(
    actuator.account_status_df["net_value"], benchmark=actuator.account_status_df["price"]["ETH"]
).items()})

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-07-22 16:24:15,340 - INFO - init strategy...
2024-07-22 16:24:15,343 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:02<00:00, 2743.58it/s]
2024-07-22 16:24:17,979 - INFO - main loop finished
2024-07-22 16:24:18,125 - INFO - Print actuator summary
2024-07-22 16:24:18,321 - INFO - Backtesting finished, execute time 2.9823007583618164s


Final account status                              
Token balance in broker       
USDC      :0                        ETH       :0.055894992              
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick   pending0      pending1        liquidity
0      200278      202501  3.1833314  0.0020557076  408404128309088
1      201328      202501      0e-10         0e-22                0

Account balance history                           
l1                  net_value tokens               market1                                                                                          price     
l2                              USDC         ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2023-08-13 00:00:00    1999.5      0 0.053866915 1899.9473    

NameError: name 'performance_metrics' is not defined

Make plot about output net value、value in position、value in account and price

In [ ]:
plot_position_return_decomposition(actuator.account_status_df, actuator.token_prices[eth.name], market_key)